In [ ]:
COLAB: bool = False
if COLAB:
  !git clone https://github.com/RubenCid35/6GSmartRRM
  !mv 6GSmartRRM/* .
  !pip install -e .

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#
import os

# simple data manipulation
import numpy  as np
import pandas as pd

# deep learning
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lrs
from   torch.utils.data import DataLoader, TensorDataset, random_split

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

from collections import defaultdict

# results logging
import wandb
wandb.login()

# progress bar
from   tqdm.notebook import tqdm, trange

# remove warnings (remove deprecated warnings)
import warnings
warnings.simplefilter('ignore')

# visualization of resultsa
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from   matplotlib.ticker import MaxNLocator
import seaborn           as sns
import plotly.express as px


# wheter we are using colab or not
if not COLAB and not os.path.exists('./data/simulations'):
    os.chdir('..')

# Simulation Settings
from g6smart.sim_config import SimConfig
from g6smart.evaluation import rate as rate
from g6smart.evaluation import rate_torch as rate_metrics
from g6smart.baseline.power import iterative_wmmse
from g6smart.baseline.subband import sisa_algoritm, cgc_algoritm
config = SimConfig(0)
config

In [ ]:
# Moung Google Drive Code
if COLAB:
    drive.mount('/content/drive')

    # Move Simulations to avoid cluttering the drive folder
    if not os.path.exists('/content/simulations'):
      os.mkdir('/content/simulations')

    print("simulations folder:", list(os.listdir('/content/simulations')))
    if len(os.listdir('/content/simulations')) == 0:
      copy_tree('/content/drive/MyDrive/TFM/simulations', '/content/simulations')

    # unzip all simulations
    print("Name of the already simulated data: \n", )
    for zip_file in os.listdir('/content/simulations'):
        if zip_file.endswith('.zip'):
            print(" ----> " + zip_file)
            with zipfile.ZipFile("/content/simulations/" + zip_file, 'r') as zip_ref:
                zip_ref.extractall('/content/simulations/')

    SIMULATIONS_PATH: str = "/content/simulations"
    MODELS_PATH: str = "/content/drive/MyDrive/TFM/models/"
else:
    if not os.path.exists('./data/simulations'): os.mkdir('./data/simulations')
    for zip_file in os.listdir('data'):
        if zip_file.endswith('.zip'):
            print(" ----> " + zip_file)
            with zipfile.ZipFile("./data/" + zip_file, 'r') as zip_ref:
                zip_ref.extractall('./data/simulations')
    SIMULATIONS_PATH: str = "./data/simulations"
    MODELS_PATH: str = "./models/"
    if not os.path.exists(MODELS_PATH):
      os.mkdir(MODELS_PATH)

In [ ]:
cmg   = np.load(SIMULATIONS_PATH + '/Channel_matrix_gain.npy')
sisa_alloc = np.load(SIMULATIONS_PATH + '/sisa-allocation.npy')

# get sample from all
n_sample = 130_000
cmg   = cmg[:n_sample]
sisa_alloc = sisa_alloc[:n_sample].astype(int)

n_sample = B = cmg.shape[0]
K, N, _  = cmg.shape[1:]

shape    = lambda s: " x".join([f"{d:3d}" for d in s])
print(f"channel    matrix shape: {shape(cmg.shape)} \nallocation matrix shape: {shape(sisa_alloc.shape)}")

In [ ]:
import random
from deap import base, creator, tools, algorithms

def optimize_power_allocation(config, gain, salloc, num_individuals=50, 
                             num_generations=100, cxpb=0.7, mutpb=0.2):
    """
    Optimize power allocation using a genetic algorithm.
    
    Parameters:
    - config: Network configuration
    - gain: Gain parameters
    - salloc: Subcarrier allocation
    - num_individuals: Population size
    - num_generations: Number of generations
    - cxpb: Crossover probability
    - mutpb: Mutation probability
    
    Returns:
    - Best individual (power allocation) found
    - Minimum bit rate achieved
    """
    
    # Problem dimensions - adjust based on your network size
    num_power_variables = 20  # Example - number of power variables to optimize
    
    # Define the fitness and individual types
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)
    
    # Initialize toolbox
    toolbox = base.Toolbox()
    
    # Attribute generator - power values between 1e-5 and 1e-3
    toolbox.register("attr_float", random.uniform, 1e-5, 1e-3)
    
    # Structure initializers
    toolbox.register("individual", tools.initRepeat, creator.Individual, 
                    toolbox.attr_float, n=num_power_variables)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    
    # Evaluation function
    def evaluate(individual):
        power = np.array(individual)
        return (rate.bit_rate(config, gain, salloc, power),)
    
    toolbox.register("evaluate", evaluate)
    toolbox.register("mate", tools.cxBlend, alpha=0.5)  # Blend crossover
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1e-6, indpb=0.1)
    toolbox.register("select", tools.selTournament, tournsize=3)
    
    # Create population
    population = toolbox.population(n=num_individuals)
    
    # Track best individual
    hof = tools.HallOfFame(1)
    
    # Run the algorithm
    algorithms.eaSimple(population, toolbox, cxpb=cxpb, mutpb=mutpb,
                        ngen=num_generations, halloffame=hof, verbose=True)
    
    # Return best solution
    best_individual = hof[0]
    best_fitness = evaluate(best_individual)[0]
    
    return best_individual, best_fitness


In [ ]:
gain  = cmg[1]

# subband allocation
calloc = cgc_algoritm(gain, 4)
salloc = sisa_algoritm(gain, 50)[0]

print("GCG  Allocation:\n", list(calloc))
print("SISA Allocation:\n", list(salloc))

# compare bitrates
bitrates = {}
# no allocation
mp = config.transmit_power
for k in range(K):
    alloc = np.full((N, ), k, dtype = np.int64)
    brate = rate.bit_rate(config, gain, alloc, mp).mean()
    bitrates[f"Band {k} (FP)"] = brate

# cgc alloc
bitrates[f"CGC {k} (FP)"] = rate.bit_rate(config, gain, calloc, mp).mean()
bitrates[f"SISA {k} (FP)"] = rate.bit_rate(config, gain, salloc, mp).mean()

plt.bar(bitrates.keys(), bitrates.values())
plt.xticks(rotation = 45);


In [ ]:
calloc = cgc_algoritm(gain, 4)
salloc = sisa_algoritm(gain, 50)[0]

_, ax = plt.subplots(1, 2, figsize = (12, 4))
for i, mp in enumerate([1.e-3, 1.2e-3]):
    print(mp)

    bitrates = {}
    # no allocation
    for k in range(K):
        alloc  = np.full((N, ), k, dtype = np.int64)
        brate1 = rate.bit_rate(config, gain, alloc, mp).min()
        bitrates[f"Band {k} (FP)"] = brate1

        power  = iterative_wmmse(gain, alloc, mp, config.noise_power, 10000, 1e-10)
        brate2 = rate.bit_rate(config, gain, alloc, power).min()
        bitrates[f"Band {k} (WMMSE)"] = brate2

    bitrates[f"CGC {k} (FP)"] = rate.bit_rate(config, gain, calloc, mp).min()
    power  = iterative_wmmse(gain, calloc, mp, config.noise_power, 50, 1e-10)
    bitrates[f"CGC {k} (WMMSE)"] = rate.bit_rate(config, gain, calloc, power).min()

    bitrates[f"SISA {k} (FP)"] = rate.bit_rate(config, gain, salloc, mp).min()
    power  = iterative_wmmse(gain, salloc, mp, config.noise_power, 50, 1e-10)
    bitrates[f"SISA {k} (WMMSE-50)"] = rate.bit_rate(config, gain, salloc, power).min()

    bars = ax[i].bar(bitrates.keys(), bitrates.values())
    ax[i].tick_params(axis='x', labelrotation=90);

    ax[i].set_title(f"Comparison Methods: P_max = {i} dbm")
    for bar in bars:
        yval = bar.get_height()
        ax[i].text(bar.get_x(), yval + .5, round(yval, 1))

    ax[i].set_ylim(0, bar.get_height() + 5)


In [ ]:
R = 600
mp = 1e-1
rates = []
alloc = np.full(N, 2, dtype = np.int64)
for i in range(R):
    power  = np.random.random((N, )) * config.transmit_power
    brate2 = rate.bit_rate(config, gain, alloc, power).mean()
    rates.append(brate2)

t = rate.bit_rate(config, gain, alloc, mp).mean()
_, ax = plt.subplots(1, 1, figsize = (12, 4))
ax.plot(np.arange(R), rates)
ax.hlines(t, 0, R, colors = 'r', linestyles = "--")
plt.show()